In [113]:
import psycopg2
import pandas as pd 
import numpy as np

In [44]:
def create_connection(database, password, host="localhost",port="5432", user="postgres"):
    """"
    Description: This function create a postgresql connection object
    @ input params: database name,  password, and default values
    @ output params: create a connection
    """

    connection = None
    try:
        print('Connecting to the postgreSQL database ...')
        connection = psycopg2.connect(
            host=host, 
            port=port,
            database=database,
            user=user,
            password=password
        )
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if connection is not None:
            # connection.close()
            print('Database connection created.')
    
    return connection

In [45]:
def load_data(connection, sql_query, values=None):
    """"
    Description: This function runs the data pull from postgresql and save it as a dataframe 
    @ input params: postgresql connection object, sql query to run 
    @ output params: return a dataframe
    """
    try:
        # create a cursor 
        cursor = connection.cursor()
        cursor.execute(sql_query, values) # None
        df = pd.DataFrame(cursor.fetchall())
        df.columns = [x[0] for x in cursor.description]
        cursor.close()
        return df
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if connection is not None:
            connection.commit()
            connection.close()
            print('Database connection terminated.')

    # return df 

In [48]:
schema = "public"
table_name = "actor"
sql_select = "SELECT * FROM {}.{}".format(schema, table_name) # public.aws
conn = create_connection("users", "postgres") ## just create the connection to postgresql 
psg_df = load_data(connection=conn, sql_query=sql_select)

Connecting to the postgreSQL database ...
Database connection created.
Database connection terminated.


In [63]:
psg_df.head()

,actor_id,first_name,last_name,last_update
0,1,Penelope,Guiness,2013-05-26 14:47:57.620
1,2,Nick,Wahlberg,2013-05-26 14:47:57.620
2,3,Ed,Chase,2013-05-26 14:47:57.620
3,4,Jennifer,Davis,2013-05-26 14:47:57.620
4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620


In [99]:
psg_df.groupby(['first_name']).size().reset_index(name='counts')

,first_name,counts
0,Adam,2
1,Al,1
2,Alan,1
3,Albert,2
4,Alec,1
...,...,...
123,Whoopi,1
124,Will,1
125,William,1
126,Woody,2


In [100]:
psg_df.groupby('first_name', as_index=False).size()

,first_name,size
0,Adam,2
1,Al,1
2,Alan,1
3,Albert,2
4,Alec,1
...,...,...
123,Whoopi,1
124,Will,1
125,William,1
126,Woody,2


```python
(df
   .groupby(['col1', 'col2'])
   .agg({
      'col3': ['mean', 'count'], 
      'col4': ['median', 'min', 'count']
   }))
```

In [ ]:
psg_df.groupby(['first_name'], )

### # 01: Pandas `count()`
The pandas `.count()` function counts the number of values in each column. In the case of the psg_df dataset, there were 4 columns, and each of them had 200 values in it.

In [64]:
psg_df.count()

actor_id       200
first_name     200
last_name      200
last_update    200
dtype: int64

If you want to make your output clearer, you can select the `first_name` column first by using one of the selection operators (that we learned about in the previous article). Something like this:

In [67]:
psg_df['first_name'].count() # or 
# psg_df.first_name.count()

200

### # 02: Pandas `sum()`

Following the same logic, you can easily sum the values in the actor_id column by typing

In [68]:
psg_df['actor_id'].sum()
# psg_df.actor_id.sum()

20100

Just out of curiosity, let’s run our .sum() function on all columns, as well:

In [69]:
psg_df.sum()

/var/folders/q4/y7xqp5c55f3c58vcqh3nb1q80000gn/T/ipykernel_48153/1469557377.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  psg_df.sum()


actor_id                                                  20100
first_name    PenelopeNickEdJenniferJohnnyBetteGraceMatthewJ...
last_name     GuinessWahlbergChaseDavisLollobrigidaNicholson...
dtype: object

### Pandas Data Aggregation #03 and #04: `min()` and `max()`

How to make pandas return the smallest value from the water_need column?

In [70]:
psg_df['actor_id'].min() #or 
# psg_df.actor_id.min()

1

In [71]:
psg_df['actor_id'].max() #or 
# psg_df.actor_id.max()

200

### #05: averages in pandas: `mean()` and `median()`  
Eventually, let’s calculate statistical averages, like mean and median!

The syntax is the same as it was with the other aggregation methods above:


In [72]:
psg_df['actor_id'].mean() #or 
# psg_df.actor_id.mean()

100.5

In [73]:
psg_df['actor_id'].median() #or 
# psg_df.actor_id.median()

100.5

Okay, this was easy, right? Pandas aggregation methods are much, much easier than SQL’s, for instance.  

So it’s time to spice this up — with a little bit of grouping! Introducing the `groupby()` function!

### The pandas groupby() function (aka. grouping in pandas)  
Just as before, pandas automatically runs the .mean() calculation for every column (the first_name column disappeared since that was the column we grouped by). You can either ignore the other columns or you can remove it afterward by using one of these syntaxes:

psg_df.groupby('first_name').mean()[['col_name']] –» This returns a DataFrame object.

psg_df.groupby('first_name').mean().col_name –» This returns a Series object.

In [84]:
psg_df.groupby('first_name').mean().actor_id # ->> this returns a Series object

first_name
Adam       101.5
Al         165.0
Alan       173.0
Albert     135.5
Alec        29.0
           ...  
Whoopi     140.0
Will       168.0
William    175.0
Woody       55.0
Zero        11.0
Name: actor_id, Length: 128, dtype: float64

In [87]:
psg_df.groupby('first_name').mean()[['actor_id']] # ->> this returns a DataFrame object

,actor_id
first_name,
Adam,101.5
Al,165.0
Alan,173.0
Albert,135.5
Alec,29.0
...,...
Whoopi,140.0
Will,168.0
William,175.0


In [80]:
pd.DataFrame(psg_df.first_name.value_counts())
# df.value_counts(subset=['col1', 'col2'])

,first_name
Penelope,4
Julia,4
Kenneth,4
Burt,3
Groucho,3
...,...
Henry,1
Dustin,1
Jude,1
Carmen,1


In [89]:
psg_df.groupby('first_name').count()[['actor_id']]

,actor_id
first_name,
Adam,2
Al,1
Alan,1
Albert,2
Alec,1
...,...
Whoopi,1
Will,1
William,1


In [94]:
psg_df[psg_df.first_name == 'Penelope'].groupby(['first_name', 'last_name']).count()[['actor_id']]

actor_id
first_name last_name          
Penelope   Cronyn            1
           Guiness           1
           Monroe            1
           Pinkett           1

```sql 
--##
SELECT
	customer_id,
	Total_spend,
	CASE WHEN Total_spend <= 30 THEN 'Bronze'
		 WHEN Total_spend <= 60 THEN 'Silver'
		 WHEN Total_spend <= 90 THEN 'Gold'
		 ELSE  'Diamon'
	END AS Customer_grading
FROM (
	SELECT customer_id, SUM(amount) AS Total_spend
	FROM payment
	GROUP BY customer_id
	ORDER BY 2 ) sub ;
```

In [158]:
schema = "public"
table_name = "payment"
sql_select = "SELECT * FROM {}.{}".format(schema, table_name) # public.aws
conn = create_connection("users", "postgres") ## just create the connection to postgresql 
psg_df = load_data(connection=conn, sql_query=sql_select)

Connecting to the postgreSQL database ...
Database connection created.
Database connection terminated.


In [159]:
psg_df.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577


```python
# Below are quick example
# convert "Fee" from string to float
df['Fee'] = df['Fee'].astype(float)
print(df.dtypes)

# Convert multiple columns
df = df.astype({'Fee':'float','Discount':'float'})

# Convert all columns to floats
df = df.astype(float)
print(df.dtypes)

# Convert numeric function string to float
df['Discount'] = pd.to_numeric(df['Discount'])
print(df.dtypes)

# Convert DataFrame column from string to float
df["Discount"] = pd.to_numeric(df["Discount"], downcast="float")
print(df.dtypes)

# Convert each value of the column to a string
df['Discount'] = pd.to_numeric(df['Discount'], errors='coerce')
print(df.dtypes)

# Using df.replace() to replace nan values 0 before convertion
df['Discount'] = pd.to_numeric(df['Discount'], errors='coerce')
df = df.replace(np.nan, 0, regex=True)
print(df)
print(df.dtypes)

# Replace empty string ('') with np.nan before convertion
df['Discount']=df.Discount.replace('',np.nan).astype(float)
print(df)
print(df.dtypes)
```

In [160]:
psg_df.dtypes
# psg_df[['customer_id','amount']].to_numeric('amount').dtypes

payment_id               int64
customer_id              int64
staff_id                 int64
rental_id                int64
amount                  object
payment_date    datetime64[ns]
dtype: object

In [161]:
# psg_df['amount'] = psg_df['amount'].astype(float)
psg_df = psg_df.astype({'amount':'float'})
psg_df.dtypes

payment_id               int64
customer_id              int64
staff_id                 int64
rental_id                int64
amount                 float64
payment_date    datetime64[ns]
dtype: object

In [130]:
psg_grouped = psg_df[['customer_id','amount']].groupby('customer_id').sum().reset_index().rename(columns={'amount':'total_spend'})
psg_grouped.head()

,customer_id,total_spend
0,1,114.70
1,2,123.74
2,3,130.76
3,4,81.78
4,5,134.65


In [131]:
psg_grouped['customer_grading'] = psg_grouped['total_spend'].map(lambda x: 'Bronze' if x <= 30 else ('Silver' if x <= 60 else ('Gold' if x<= 90 else 'Dimond')))

In [132]:
psg_grouped.head()

,customer_id,total_spend,customer_grading
0,1,114.70,Dimond
1,2,123.74,Dimond
2,3,130.76,Dimond
3,4,81.78,Gold
4,5,134.65,Dimond


In [133]:
# simpler way 
def customer_grading(val):
    result = None
    if val <= 30:
        result = "Bronze"
    elif val <= 60:
        result = "Silver"
    elif val <= 90:
        result = "Gold"
    else:
        result = "Dimond"
    return result

In [140]:
psg_grouped['customer_grading2'] = psg_grouped['total_spend'].apply(lambda x: customer_grading(x)) 

In [141]:
psg_grouped.head()

,customer_id,total_spend,customer_grading,customer_grading2
0,1,114.70,Dimond,Dimond
1,2,123.74,Dimond,Dimond
2,3,130.76,Dimond,Dimond
3,4,81.78,Gold,Gold
4,5,134.65,Dimond,Dimond


In [148]:
pd.DataFrame(psg_grouped.customer_grading.value_counts()).reset_index().rename(columns={'index':'customer_grading', 'customer_grading':'count'})

,customer_grading,count
0,Dimond,394
1,Gold,188
2,Silver,16
3,Bronze,1


In [143]:
psg_grouped.customer_grading2.value_counts()

Dimond    394
Gold      188
Silver     16
Bronze      1
Name: customer_grading2, dtype: int64

```sql
--Common Table Expressions (CTE)
WITH sub AS (SELECT customer_id, SUM(amount) AS Total_spend
	FROM payment
	GROUP BY customer_id
	ORDER BY 2 ),
customer_grading AS (
SELECT
	customer_id,
	Total_spend,
	CASE WHEN Total_spend <= 30 THEN 'Bronze'
		 WHEN Total_spend <= 60 THEN 'Silver'
		 WHEN Total_spend <= 90 THEN 'Gold'
		 ELSE  'Diamon'
	END AS Customer_grading
FROM sub)
SELECT Customer_grading, COUNT(Customer_grading)
FROM customer_grading
GROUP BY Customer_grading
ORDER BY 2
;
SELECT *
FROM actor_info;
```

```sql 
---## WINDOW FUNCTIONS
SELECT *
	--customer_id, DATE(payment_date)
	, SUM(amount) OVER (PARTITION BY customer_id ) AS Total_spend
FROM payment
ORDER BY customer_id DESC;
SELECT customer_id
	 , amount
-- 	--, DATE(payment_date)
	, SUM(amount) OVER ( ORDER BY customer_id ) AS Total_spend
	
	, ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY amount) AS Row_no
	, RANK() OVER (PARTITION BY customer_id ORDER BY amount) AS Rk
	, DENSE_RANK() OVER (PARTITION BY customer_id ORDER BY amount) AS Dr
FROM payment
--ORDER BY DATE(payment_date) DESC;
SELECT *
FROM payment;
```

In [283]:
test_df = psg_df.copy()
print(test_df.shape)
test_df.head()

(14596, 6)


,payment_id,customer_id,staff_id,rental_id,amount,payment_date
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577


In [193]:
result_df = test_df.groupby(['customer_id']).apply(lambda gdf: gdf.assign(total_spend = lambda gdf: gdf['amount'].sum()))#.droplevel(0)

In [194]:
result_df.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,total_spend
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,103.78
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577,103.78
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,103.78
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577,103.78
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,103.78


In [238]:
# test_df.groupby(['customer_id']).apply(lambda x: x.sort_values(['amount'], ascending=False)).reset_index(drop=True)
# df.groupby(["name"]).apply(lambda x: x.sort_values(["count_1"], ascending = False)).reset_index(drop=True) # cumcount()+1
second_result = test_df.groupby(['customer_id']).apply(lambda x: x.assign(rank = lambda x: x.groupby(['customer_id']).cumcount() + 1))
# df.groupby(['c1', 'c2']).cumcount()+1 

In [239]:
second_result.shape

(14596, 7)

In [240]:
second_result[second_result['customer_id'] == 264]

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,rank
1995,19498,264,2,1165,3.99,2007-02-14 21:44:53.996577,1
1996,19499,264,1,1206,4.99,2007-02-15 00:55:33.996577,2
1997,19500,264,1,3028,0.99,2007-02-20 10:19:18.996577,3
1998,19501,264,1,3403,3.99,2007-02-21 14:23:32.996577,4
7625,25128,264,2,10792,2.99,2007-03-01 20:12:50.996577,5
7626,25129,264,1,11527,3.99,2007-03-16 22:53:32.996577,6
7627,25130,264,2,11533,0.99,2007-03-16 23:03:19.996577,7
7628,25131,264,1,11539,2.99,2007-03-16 23:14:07.996577,8
7629,25132,264,1,12518,4.99,2007-03-18 12:09:58.996577,9
7630,25133,264,2,13590,2.99,2007-03-20 04:17:25.996577,10


In [251]:
# third_result = test_df.groupby(['customer_id']).apply(lambda df: df.assign(rk = lambda df: df.groupby(['amount']).apply(pd.Series.rank).astype(int)))
# second_result = test_df.groupby(['customer_id']).apply(lambda x: x.assign(rank = lambda x: x.groupby(['customer_id']).cumcount() + 1))
# test_df.head()
# df.groupby(['companyName'])['overallRating'].sum().sort_values(ascending=False).head(20)

# ranks = scores.abs().apply(pd.Series.rank).astype(int)
third_result = test_df.groupby(['customer_id']).apply(lambda df: df.assign(rk = lambda df: df.groupby(['customer_id'])['amount'].rank("dense", ascending=False).astype(int)))
third_result.head()
# df.groupby("group_ID")["value"].rank("dense", ascending=False)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,rk
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,3
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577,8
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,3
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577,7
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,3


In [282]:
third_result[third_result['customer_id'] == 341].sort_values('amount',ascending=False)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,rk
8510,26013,341,1,8728,9.99,2007-04-29 16:41:15.996577,1
2701,20204,341,2,13074,8.99,2007-03-19 08:35:19.996577,2
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,3
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,3
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,3
2698,20201,341,1,11305,6.99,2007-03-02 14:13:21.996577,4
2705,20208,341,2,15938,6.99,2007-03-23 17:11:57.996577,4
5,17508,341,1,3382,5.99,2007-02-21 12:33:49.996577,5
2702,20205,341,2,13806,4.99,2007-03-20 11:22:12.996577,6
8504,26007,341,2,3938,4.99,2007-04-06 19:44:11.996577,6


In [280]:
fourth_result = test_df.groupby(['customer_id']).apply(lambda df: df.assign(rk = lambda df: df.groupby(['customer_id'])['amount'].rank("min", ascending=False).astype(int)))
fourth_result.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,rk
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,3
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577,18
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,3
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577,13
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,3


In [277]:
fourth_result[(fourth_result['customer_id'] == 341)].amount.value_counts().sort_index(ascending=False)

9.99    1
8.99    1
7.99    3
6.99    2
5.99    1
4.99    4
2.99    5
1.99    1
0.99    4
Name: amount, dtype: int64

In [281]:
fourth_result[(fourth_result['customer_id'] == 341)].sort_values(by=['amount'], ascending=False)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,rk
8510,26013,341,1,8728,9.99,2007-04-29 16:41:15.996577,1
2701,20204,341,2,13074,8.99,2007-03-19 08:35:19.996577,2
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,3
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,3
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,3
2698,20201,341,1,11305,6.99,2007-03-02 14:13:21.996577,6
2705,20208,341,2,15938,6.99,2007-03-23 17:11:57.996577,6
5,17508,341,1,3382,5.99,2007-02-21 12:33:49.996577,8
2702,20205,341,2,13806,4.99,2007-03-20 11:22:12.996577,9
8504,26007,341,2,3938,4.99,2007-04-06 19:44:11.996577,9


In [288]:
def dens_rank(df):
    return df.groupby(['customer_id'])['amount'].rank('dense', ascending=False).astype(int)

def rank(df):
    return df.groupby(['customer_id'])['amount'].rank("min", ascending=False).astype(int)

def row_number(df):
    return df.groupby(['customer_id']).cumcount() + 1

def add_sum(df):
    return df['amount'].sum()

In [290]:
sum_df = test_df.groupby(['customer_id']).apply(lambda df: df.assign(total_spend = lambda df: add_sum(df)))#.droplevel(0)

In [291]:
sum_df.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,total_spend
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,103.78
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577,103.78
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,103.78
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577,103.78
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,103.78


In [292]:
## ROW_NUMBER()
row_number_df = test_df.groupby(['customer_id']).apply(lambda df: df.assign(row_number = lambda df: row_number(df)))
row_number_df.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,row_number
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,1
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577,2
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,3
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577,4
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,5


In [294]:
## RANK()
rank_df = test_df.groupby(['customer_id']).apply(lambda df: df.assign(rank = lambda df: rank(df)))
rank_df.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,rank
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,3
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577,18
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,3
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577,13
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,3


In [295]:
## DENSE_RANK()
dense_rank_df = test_df.groupby(['customer_id']).apply(lambda df: df.assign(dense_rank = lambda df: dens_rank(df)))
dense_rank_df.head()


,payment_id,customer_id,staff_id,rental_id,amount,payment_date,dense_rank
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,3
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577,8
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,3
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577,7
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,3


In [297]:
dense_rank_df[(dense_rank_df['customer_id'] == 341)].sort_values(by=['amount'], ascending=False)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,dense_rank
8510,26013,341,1,8728,9.99,2007-04-29 16:41:15.996577,1
2701,20204,341,2,13074,8.99,2007-03-19 08:35:19.996577,2
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,3
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,3
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,3
2698,20201,341,1,11305,6.99,2007-03-02 14:13:21.996577,4
2705,20208,341,2,15938,6.99,2007-03-23 17:11:57.996577,4
5,17508,341,1,3382,5.99,2007-02-21 12:33:49.996577,5
2702,20205,341,2,13806,4.99,2007-03-20 11:22:12.996577,6
8504,26007,341,2,3938,4.99,2007-04-06 19:44:11.996577,6
